# TODO Compared with from scratch standard training and attacks 

In [1]:
%load_ext autoreload
%autoreload 2


In [2]:
#TransferLearning of ATTACK --> 
# VS From Scratch
import torch
import torchvision
import torchvision.transforms as transforms
from utils import *
import torch.optim as optim
from model.baseline_model import *
from trainer import *
from torch.utils.tensorboard import SummaryWriter
import robustbench
#from robustbench.data import load_cifar10
from robustbench.utils import load_model, clean_accuracy
from finetune_framework import *

import warnings
warnings.filterwarnings('ignore')

PyTorch Version:  1.7.1+cu110
Torchvision Version:  0.8.2+cu110


In [11]:
model_name = "resnet"
num_classes = 10
feature_extract = False

model_scratch, input_size = initialize_model(model_name, num_classes, feature_extract, use_pretrained=False)
train_dataset, val_dataset, train_loader, val_loader = CIFAR10(128, finetune = True, 
                                                               input_size = input_size, test_batch_size=128)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
device = "cuda"

Files already downloaded and verified
Files already downloaded and verified


In [12]:
model_scratch = model_scratch.to(device)
params_to_update = model_scratch.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_scratch = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [13]:
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()
PATH = './cifar_resnet_scratch.pth'
device = "cuda"
from torch.utils.tensorboard import SummaryWriter
writer = SummaryWriter("Standard_Resnet_Scratch")

In [14]:
train(model_scratch, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_scratch, 100, writer, 
                 PATH, False, None, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 1.920101451751826	 lr: 0.001
Epoch [1/100]	 Validation Loss: 1.7191996257516402	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 1.7279964761660838	 lr: 0.001
Epoch [2/100]	 Validation Loss: 1.5322630586503427	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 1.6405305661203917	 lr: 0.001
Epoch [3/100]	 Validation Loss: 1.460173717027978	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 1.5757409812849197	 lr: 0.001
Epoch [4/100]	 Validation Loss: 1.4013633335692972	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 1.5157668130172184	 lr: 0.001
Epoch [5/100]	 Validation Loss: 1.3420712057548234	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 1.4632819232428471	 lr: 0.001
Epoch [6/100]	 Validation Loss: 1.2792894063116629	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 1.4183214271769804	 lr: 0.001
Epoch [7/100]	 Validation Loss: 1.2978188508673558	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss

Epoch [60/100]	 Training Loss: 0.6679742376670204	 lr: 0.001
Epoch [60/100]	 Validation Loss: 0.5474943582770191	 lr: 0.001
Epoch [60/100]	
Epoch [61/100]	 Training Loss: 0.6557074269980115	 lr: 0.001
Epoch [61/100]	 Validation Loss: 0.5204248201997974	 lr: 0.001
Epoch [61/100]	
Epoch [62/100]	 Training Loss: 0.6617547073175231	 lr: 0.001
Epoch [62/100]	 Validation Loss: 0.5567510067662106	 lr: 0.001
Epoch [62/100]	
Epoch [63/100]	 Training Loss: 0.6529458463191986	 lr: 0.001
Epoch [63/100]	 Validation Loss: 0.520437446198886	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 0.6516531833145015	 lr: 0.001
Epoch [64/100]	 Validation Loss: 0.5944521842123587	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 0.648900988233059	 lr: 0.001
Epoch [65/100]	 Validation Loss: 0.5386933405188066	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 0.6415800507873526	 lr: 0.001
Epoch [66/100]	 Validation Loss: 0.5405959196483032	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Traini

In [15]:
print("the clean accuracy is ")
test(val_loader, model_scratch)

the clean accuracy is 
Accuracy of the network on the 10000 test images: 85 %


In [16]:
print("PGD")
pgd_attack_transfer = generate_adv(model_scratch, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, model_scratch, "cuda")

PGD
[Model loaded]
Acc: 4.20 %
adv model


In [17]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_scratch, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, model_scratch, "cuda")

fgsm
[Model loaded]
Acc: 25.06 %
adv model


In [18]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_scratch, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, model_scratch, "cuda")

nifgsm
[Model loaded]
Acc: 9.46 %
adv model


In [19]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_scratch, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, model_scratch, "cuda")

vmifgsm
[Model loaded]
Acc: 4.59 %
adv model


# TODO Compared with from scratch adversarial training and attacks 

In [20]:
from torch.utils.tensorboard import SummaryWriter
import copy
net_scratch_adv = copy.deepcopy(model_scratch) #am I doing correct? 
# for param in net_adv.parameters():
#     print(param)

In [21]:
params_to_update = net_scratch_adv.parameters()

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in net_scratch_adv.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_adv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()

PATH = './cifar_trans_resnet_scratch_adv_pgd.pth'
device = "cuda"
writer = SummaryWriter("Adv_trans_resnet_scratch_PGD")
pgd_attack = generate_adv(net_scratch_adv, "pgd")

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

In [22]:
train(net_scratch_adv, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_adv, 100, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

Epoch [0/100]	
Epoch [1/100]	 Training Loss: 2.0469779568864865	 lr: 0.001
Epoch [1/100]	 Validation Loss: 12.82632237446459	 lr: 0.001
Epoch [1/100]	
Epoch [2/100]	 Training Loss: 1.7371841123341905	 lr: 0.001
Epoch [2/100]	 Validation Loss: 26.673698908166042	 lr: 0.001
Epoch [2/100]	
Epoch [3/100]	 Training Loss: 1.6720261217078285	 lr: 0.001
Epoch [3/100]	 Validation Loss: 13.963512686234486	 lr: 0.001
Epoch [3/100]	
Epoch [4/100]	 Training Loss: 1.629674499297081	 lr: 0.001
Epoch [4/100]	 Validation Loss: 12.616864958895913	 lr: 0.001
Epoch [4/100]	
Epoch [5/100]	 Training Loss: 1.6080426020390541	 lr: 0.001
Epoch [5/100]	 Validation Loss: 15.755662688726112	 lr: 0.001
Epoch [5/100]	
Epoch [6/100]	 Training Loss: 1.5803513456793392	 lr: 0.001
Epoch [6/100]	 Validation Loss: 17.246170913116842	 lr: 0.001
Epoch [6/100]	
Epoch [7/100]	 Training Loss: 1.5713662128619221	 lr: 0.001
Epoch [7/100]	 Validation Loss: 11.563516133948218	 lr: 0.001
Epoch [7/100]	
Epoch [8/100]	 Training Loss

Epoch [63/100]	 Training Loss: 1.3129097150109919	 lr: 0.001
Epoch [63/100]	 Validation Loss: 46.63334095628956	 lr: 0.001
Epoch [63/100]	
Epoch [64/100]	 Training Loss: 1.3133342750846881	 lr: 0.001
Epoch [64/100]	 Validation Loss: 47.947365917736974	 lr: 0.001
Epoch [64/100]	
Epoch [65/100]	 Training Loss: 1.3118542792547085	 lr: 0.001
Epoch [65/100]	 Validation Loss: 42.16519580309904	 lr: 0.001
Epoch [65/100]	
Epoch [66/100]	 Training Loss: 1.3070138555658444	 lr: 0.001
Epoch [66/100]	 Validation Loss: 43.04703598988207	 lr: 0.001
Epoch [66/100]	
Epoch [67/100]	 Training Loss: 1.3066618143749968	 lr: 0.001
Epoch [67/100]	 Validation Loss: 44.80973444105704	 lr: 0.001
Epoch [67/100]	
Epoch [68/100]	 Training Loss: 1.29853588083516	 lr: 0.001
Epoch [68/100]	 Validation Loss: 50.18283187286763	 lr: 0.001
Epoch [68/100]	
Epoch [69/100]	 Training Loss: 1.3007290512704484	 lr: 0.001
Epoch [69/100]	 Validation Loss: 45.95871411094183	 lr: 0.001
Epoch [69/100]	
Epoch [70/100]	 Training Los

In [23]:
print("the clean accuracy is ")
test(val_loader, net_scratch_adv)

the clean accuracy is 
Accuracy of the network on the 10000 test images: 11 %


In [24]:
print("PGD")
pgd_attack_transfer = generate_adv(model_scratch, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, net_scratch_adv, "cuda")

PGD
[Model loaded]
Acc: 74.38 %
adv model


In [25]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_scratch, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, net_scratch_adv, "cuda")

fgsm
[Model loaded]
Acc: 74.31 %
adv model


In [26]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_scratch, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, net_scratch_adv, "cuda")

nifgsm
[Model loaded]
Acc: 74.03 %
adv model


In [27]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_scratch, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, net_scratch_adv, "cuda")

vmifgsm
[Model loaded]
Acc: 74.00 %
adv model


##### Prevent overfiting 

In [11]:
model_load = torch.load("./cifar_resnet_scratch.pt")
model_scratch = torch.load("./cifar_resnet_scratch.pt")


In [12]:
model_load

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [14]:
# model_name = "resnet"
# num_classes = 10
# feature_extract = False


# train_dataset, val_dataset, train_loader, val_loader = CIFAR10(128, finetune = True, 
#                                                                input_size = input_size, test_batch_size=128)
# classes = ('plane', 'car', 'bird', 'cat',
#            'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
# device = "cuda"

NameError: name 'input_size' is not defined

In [15]:
params_to_update = model_load.parameters()

feature_extract = False

print("Params to learn:")
if feature_extract:
    params_to_update = []
    for name,param in model_load.named_parameters():
        if param.requires_grad == True:
            params_to_update.append(param)
            print("\t",name)
else:
    for name,param in model_scratch.named_parameters():
        if param.requires_grad == True:
            print("\t",name)

# Observe that all parameters are being optimized
optimizer_adv = optim.SGD(params_to_update, lr=0.001, momentum=0.9)
train_criterion = nn.CrossEntropyLoss()
val_criterion = nn.CrossEntropyLoss()

PATH = './cifar_trans_resnet_scratch_adv_pgd_10.pth'
device = "cuda"
writer = SummaryWriter("Adv_trans_resnet_scratch_PGD_10")
pgd_attack = generate_adv(model_load, "pgd")

Params to learn:
	 conv1.weight
	 bn1.weight
	 bn1.bias
	 layer1.0.conv1.weight
	 layer1.0.bn1.weight
	 layer1.0.bn1.bias
	 layer1.0.conv2.weight
	 layer1.0.bn2.weight
	 layer1.0.bn2.bias
	 layer1.1.conv1.weight
	 layer1.1.bn1.weight
	 layer1.1.bn1.bias
	 layer1.1.conv2.weight
	 layer1.1.bn2.weight
	 layer1.1.bn2.bias
	 layer2.0.conv1.weight
	 layer2.0.bn1.weight
	 layer2.0.bn1.bias
	 layer2.0.conv2.weight
	 layer2.0.bn2.weight
	 layer2.0.bn2.bias
	 layer2.0.downsample.0.weight
	 layer2.0.downsample.1.weight
	 layer2.0.downsample.1.bias
	 layer2.1.conv1.weight
	 layer2.1.bn1.weight
	 layer2.1.bn1.bias
	 layer2.1.conv2.weight
	 layer2.1.bn2.weight
	 layer2.1.bn2.bias
	 layer3.0.conv1.weight
	 layer3.0.bn1.weight
	 layer3.0.bn1.bias
	 layer3.0.conv2.weight
	 layer3.0.bn2.weight
	 layer3.0.bn2.bias
	 layer3.0.downsample.0.weight
	 layer3.0.downsample.1.weight
	 layer3.0.downsample.1.bias
	 layer3.1.conv1.weight
	 layer3.1.bn1.weight
	 layer3.1.bn1.bias
	 layer3.1.conv2.weight
	 layer3.1.b

2022-12-17 03:29:28.589432: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-17 03:29:28.868745: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-12-17 03:29:29.950203: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/cuda/lib64:/usr/local/nccl2/lib:/usr/local/cuda/extras/CUPTI/lib64
2022-12-17 03:29:29.950379: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer

In [16]:
train(model_load, train_loader, train_criterion, val_loader, val_criterion, 
              optimizer_adv, 10, writer, 
                 PATH, True, pgd_attack, device="cuda",
                  n_steps_show=1000)

NameError: name 'train_loader' is not defined

In [23]:
print("the clean accuracy is ")
test(val_loader, model_load)

the clean accuracy is 
Accuracy of the network on the 10000 test images: 11 %


In [24]:
print("PGD")
pgd_attack_transfer = generate_adv(model_scratch, "pgd")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

pgd_acc = test_attack(val_loader, pgd_attack_transfer, model_load, "cuda")

PGD
[Model loaded]
Acc: 74.38 %
adv model


In [25]:
print("fgsm")
fgsm_attack_transfer = generate_adv(model_scratch, "fgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

fgsm_acc = test_attack(val_loader, fgsm_attack_transfer, model_load, "cuda")

fgsm
[Model loaded]
Acc: 74.31 %
adv model


In [26]:
print("nifgsm")
nifgsm_attack_transfer = generate_adv(model_scratch, "nifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

nifgsm_acc = test_attack(val_loader, nifgsm_attack_transfer, model_load, "cuda")

nifgsm
[Model loaded]
Acc: 74.03 %
adv model


In [27]:
print("vmifgsm")
vmifgsm_attack_transfer = generate_adv(model_scratch, "vmifgsm")
# pgd_attack_transfer = pgd_attack_transfer.cpu()

vmifgsm_acc = test_attack(val_loader, vmifgsm_attack_transfer, model_load, "cuda")

vmifgsm
[Model loaded]
Acc: 74.00 %
adv model
